In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CS131_A4_DecisionTree").getOrCreate() 


25/04/19 01:51:48 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
csv_path = "gs://dataproc-staging-us-central1-1084640347746-mwcssbrt/data/2019-01-h1.csv"  # update to your actual GCS path
df = spark.read.csv(csv_path, header=True, inferSchema=True)


In [9]:
# Select required columns and filter out invalid rows (e.g., passenger_count == 0.0)
selected = df.select("passenger_count", "pulocationid", "dolocationid", "total_amount") \
             .dropna() \
             .filter("passenger_count > 0")

selected.show(10)


+---------------+------------+------------+------------+
|passenger_count|pulocationid|dolocationid|total_amount|
+---------------+------------+------------+------------+
|            1.0|       151.0|       239.0|        9.95|
|            1.0|       239.0|       246.0|        16.3|
|            3.0|       236.0|       236.0|         5.8|
|            5.0|       193.0|       193.0|        7.55|
|            5.0|       193.0|       193.0|       55.55|
|            5.0|       193.0|       193.0|       13.31|
|            5.0|       193.0|       193.0|       55.55|
|            1.0|       163.0|       229.0|        9.05|
|            1.0|       229.0|         7.0|        18.5|
|            2.0|       141.0|       234.0|        13.0|
+---------------+------------+------------+------------+
only showing top 10 rows



In [10]:
trainDF, testDF = selected.randomSplit([0.8, 0.2], seed=42)


In [11]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml import Pipeline

assembler = VectorAssembler(inputCols=["passenger_count", "pulocationid", "dolocationid"], outputCol="features")
dt = DecisionTreeRegressor(featuresCol="features", labelCol="total_amount", maxBins=100)
pipeline = Pipeline(stages=[assembler, dt])


In [13]:
model = pipeline.fit(trainDF)


In [14]:
predictions = model.transform(testDF)
predictions.select("passenger_count", "pulocationid", "dolocationid", "total_amount", "prediction").show(10)


+---------------+------------+------------+------------+------------------+
|passenger_count|pulocationid|dolocationid|total_amount|        prediction|
+---------------+------------+------------+------------+------------------+
|            1.0|         1.0|         1.0|        15.3|22.793349915525248|
|            1.0|         1.0|         1.0|       24.36|22.793349915525248|
|            1.0|         1.0|         1.0|        40.3|22.793349915525248|
|            1.0|         1.0|         1.0|        65.3|22.793349915525248|
|            1.0|         1.0|         1.0|        93.8|22.793349915525248|
|            1.0|         1.0|         1.0|       100.3|22.793349915525248|
|            1.0|         1.0|         1.0|       105.3|22.793349915525248|
|            1.0|         1.0|         1.0|      115.56|22.793349915525248|
|            1.0|         3.0|         3.0|        48.8|22.793349915525248|
|            1.0|         3.0|         3.0|        50.8|22.793349915525248|
+-----------

In [15]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="total_amount", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE):", rmse)


Root Mean Squared Error (RMSE): 10.17763249423234
